In [ ]:
#!pip install yfinance
#!pip install hmmlearn
#pip install scipy
import
from sklearn.mixture import GaussianMixture
import yfinance as yf
import numpy as np
from numpy import random
import pandas as pd
from scipy.stats import norm

In [5]:
ticker = "^GSPC"  # S&P 500 index symbol

data = yf.download(ticker, start="1950-01-01", end="2016-11-30", interval="1mo")
data.dropna(inplace=True)

[*********************100%%**********************]  1 of 1 completed


In [ ]:
descriptive_stats = {
    "Price": ["Open", "High", "Low", "Close"],
    "Min": [data['Open'].min(), data['High'].min(), data['Low'].min(), data['Close'].min()],
    "Max": [data['Open'].max(), data['High'].max(), data['Low'].max(), data['Close'].max()],
    "Mean": [data['Open'].mean(), data['High'].mean(), data['Low'].mean(), data['Close'].mean()],
    "Std.": [data['Open'].std(), data['High'].std(), data['Low'].std(), data['Close'].std()],
}

summary_table = pd.DataFrame(descriptive_stats)
summary_table.to_csv('summary_table.csv', index=False, sep=';')
print("Summary_table.csv")
print(summary_table)

   Price         Min          Max         Mean        Std.
0   Open  167.199997  2173.149902   969.192478  543.577386
1   High  180.270004  2214.100098  1000.686034  557.155179
2    Low  163.360001  2147.580078   935.889869  526.896105
3  Close  179.630005  2198.810059   974.394776  545.519109
Summary_table.csv


In [ ]:
#need to do that only for T period of time 
#dont use the stack but just closing prices
#plot the result (which should 4 states)

#Model selection 
features = data['Close'].values
features = close.reshape(-1, 1)  # Reshape to a 2D array

close = np.column_stack([
    data['Open'].fillna(0),  # Percentage change of open price
    data['High'].fillna(0),  # Percentage change of high price
    data['Low'].fillna(0),   # Percentage change of low price
    data['Close'].fillna(0)  # Percentage change of close price
])

def calculate_model_criteria(hmm_model, M, k):  # M as number of obserations, k as number of parameters 
    L = model_gm.score(close)
    aic = -2 * L + 2 * k
    bic = -2 * L + k * np.log(M)
    hqc = -2 * L + 2 * k * np.log(np.log(M))
    caic = -2 * L + k * (1 + np.log(M))
    return {"AIC": aic, "BIC": bic, "HQC": hqc, "CAIC": caic}

state_range = range(2, 7)  # HMM with 2 to 6 states
data_length = len(close)

def res (obs, state_range):
    results = []
    for num_states in state_range:
        model_gm = GaussianMixture(n_components=num_states, covariance_type="diag", random_state=42)
        model_gm.fit(close)
        num_params = num_states ** 2 + 2 * num_states - 1 
        criteria = calculate_model_criteria(model_gm, data_length, num_params)
        results.append({"States": num_states, **criteria})
    return pd.DataFrame(results)

res_df = res(close, state_range)
print(res_df)

c:\Users\graci\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


ValueError: X has 4 features, but GaussianMixture is expecting 1 features as input.

In [ ]:
# Step 6: Predict closing price
predicted_prices = []
har_model = data['Close'].rolling(window=10).mean().shift(1)

for i in range(len(close)):
    state = best_model.predict(close[:i + 1])[-1]  # Predict current state
    state_means = best_model.means_[state]            # Mean of observed values in the predicted state
    predicted_price = data['Close'].iloc[i] * (1 + state_means[-1])  # Predict close price based on % change
    predicted_prices.append(predicted_price)

In [ ]:
# Step 7: Evaluate predictions
data['Predicted_Close'] = predicted_prices
data['HAR_Close'] = har_model
data.dropna(inplace=True)

hmm_mse = mean_squared_error(data['Close'], data['Predicted_Close'])
har_mse = mean_squared_error(data['Close'], data['HAR_Close'])
print(f"HMM MSE: {hmm_mse}, HAR MSE: {har_mse}")

In [ ]:
# Step 8: Trading simulation (buy if predicted rise, sell otherwise)
data['Signal'] = np.where(data['Predicted_Close'] > data['Close'], 1, -1)
data['Return'] = data['Signal'] * data['Close'].pct_change()

hmm_cumulative_return = (1 + data['Return']).cumprod()
buy_and_hold_return = (1 + data['Close'].pct_change()).cumprod()

# Print results
print(f"HMM Cumulative Return: {hmm_cumulative_return.iloc[-1]:.2f}")
print(f"Buy-and-Hold Cumulative Return: {buy_and_hold_return.iloc[-1]:.2f}")